In [42]:
import torch
import bertviz
from bertviz import model_view, head_view
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
import sys
sys.path.append('../') # Dirty hack
from train_transformers import get_base_transformer_lm, get_datasets_and_vocab
from vocabulary import CharVocabulary
from data_utils import build_dataset_addmult_mod10

from evaluate_transformers import package_data


In [44]:
MODEL_LOAD_PATH = '/home/x11kjm/structural-grokking/checkpoints/231117_runs/state_150000.pt'

In [45]:
out_vocab = CharVocabulary(chars=set('0123456789'))
# datasets, in_vocab = get_datasets_and_vocab(, args.lm)

in_vocab = CharVocabulary(chars=set('0123456789+*()='))

vec_dim = 512
n_heads = 20
encoder_n_layers = 2

In [46]:
model, interface = get_base_transformer_lm(in_vocab, vec_dim, n_heads, encoder_n_layers)

In [50]:
data_batch = ['(*(+(+41)(*40))(*7(*(+17)8)))', '(*(*(*(*4(+9(+73)))6)0)4)']

data_batch = ['(+(+4(*32))(*41))']

data_tensor = package_data(data_batch, in_vocab)
src = data_tensor['in'].T
src_len = data_tensor['in_len']

mask = model.generate_len_mask(src.shape[1], src_len)
src = model.pos_embed(model.input_embed(src), 0)
attn_matrices = model.trafo.get_attn_matrices(
    src, tgt=None, src_length_mask=mask
)
all_tokens = list(in_vocab.to_index.keys())


layer_stack = torch.stack(attn_matrices, dim=0)

layers, batch, seq_len, _ = layer_stack.shape

layer_stack = layer_stack.reshape(batch, layers, seq_len, seq_len)


print(data_batch)


['(+(+4(*32))(*41))']


In [51]:
model_view(layer_stack.unsqueeze(0), list(data_batch[0]))

<IPython.core.display.Javascript object>

In [52]:
head_view(layer_stack.unsqueeze(0), list(data_batch[0]))

<IPython.core.display.Javascript object>

In [ ]:
def plot_attention_matrix(attention_matrix, tokens, head=None):
    """
    Plots the attention matrix for the model's heads.
    
    :param attention_matrix: A tensor of shape [num_heads, seq_len, seq_len]
                             representing the attention weights.
    :param tokens: A list of tokens corresponding to the sequence length.
    :param head: Optional; the specific head to visualize. If None, all heads are visualized.
    """
    num_heads = attention_matrix.shape[0]
    seq_len = attention_matrix.shape[1]

    if head is not None:
        heads_to_plot = [head]
    else:
        heads_to_plot = range(num_heads)

    fig, axes = plt.subplots(1, len(heads_to_plot), figsize=(15, 5))

    if len(heads_to_plot) == 1:
        axes = [axes]

    for i, ax in enumerate(axes):
        head_idx = heads_to_plot[i]
        ax.set_title(f'Head {head_idx}')

        # Convert the tensor to a numpy array after detaching it from the computation graph
        attn_data = attention_matrix[head_idx].detach().numpy()
        
        sns.heatmap(attn_data, ax=ax, cmap='viridis', xticklabels=tokens, yticklabels=tokens)
        ax.xaxis.set_tick_params(rotation=90)
        ax.yaxis.set_tick_params(rotation=0)

    plt.tight_layout()
    plt.show()

# Usage example
plot_attention_matrix(attn_matrix, tokens)

In [ ]:
plot_attention_matrix(attn_matrices, tokens)

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings

model_name = "microsoft/xtremedistil-l12-h384-uncased"  # Find popular HuggingFace models here: https://huggingface.co/models
input_text = "The cat sat on the mat"  
model = AutoModel.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
outputs = model(inputs)  # Run model
attention = outputs[-1]  # Retrieve attention from model outputs
tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
model_view(attention, tokens)  # Display model view

In [ ]:
attention[0].shape

In [ ]:
tokens